<a href="https://colab.research.google.com/github/tmd03/DL/blob/main/GPT_test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import torch
import torch.nn.functional as F
from transformers import pipeline
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments
from torch.utils.data import DataLoader, Dataset
from datasets import Dataset

In [ ]:
train_df = pd.read_csv('https://raw.githubusercontent.com/tmd03/DL_study/main/justice_train.csv')
test_df = pd.read_csv('https://raw.githubusercontent.com/tmd03/DL_study/main/justice_test.csv')

train_dataset = Dataset.from_pandas(train_df)
test_dataset = Dataset.from_pandas(test_df)

In [ ]:
model_name = 'distilbert-base-uncased-finetuned-sst-2-english'
tokenizer = AutoTokenizer.from_pretrained(model_name)

def tokenize_function(text):
    return tokenizer(text['scenario'], padding='max_length', truncation=True)

train_dataset = train_dataset.map(tokenize_function, batched=True)
test_dataset = test_dataset.map(tokenize_function, batched=True)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/629 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

Map:   0%|          | 0/21791 [00:00<?, ? examples/s]

Map:   0%|          | 0/2704 [00:00<?, ? examples/s]

In [ ]:
print(train_dataset)

Dataset({
    features: ['label', 'scenario', 'input_ids', 'attention_mask'],
    num_rows: 21791
})


In [ ]:
# 필요한 열만 남기기 (label, input_ids, attention_mask)
train_dataset = train_dataset.remove_columns(['scenario'])
test_dataset = test_dataset.remove_columns(['scenario'])
train_dataset = train_dataset.rename_column('label', 'labels')
test_dataset = test_dataset.rename_column('label', 'labels')
train_dataset.set_format('torch')
test_dataset.set_format('torch')

In [ ]:
model = AutoModelForSequenceClassification.from_pretrained(model_name)

training_args = TrainingArguments(
    output_dir='./results',
    evaluation_strategy='epoch',
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=3,
    weight_decay=0.01
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
)

trainer.train()

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1494: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Epoch,Training Loss,Validation Loss
1,0.467900,0.544114
2,0.317400,1.122028
3,0.190700,1.272642


TrainOutput(global_step=8172, training_loss=0.3509045361422232, metrics={'train_runtime': 3470.7069, 'train_samples_per_second': 18.836, 'train_steps_per_second': 2.355, 'total_flos': 8659791252338688.0, 'train_loss': 0.3509045361422232, 'epoch': 3.0})

In [ ]:
results = trainer.evaluate()
print(results)

predictions = trainer.predict(test_dataset)
pred_labels = predictions.predictions.argmax(-1)

test_df['predicted_label'] = pred_labels
print(test_df.head())

{'eval_loss': 1.2726422548294067, 'eval_runtime': 44.5052, 'eval_samples_per_second': 60.757, 'eval_steps_per_second': 7.595, 'epoch': 3.0}
   label                                           scenario  predicted_label
0      1  I used to walk around the park with my coworke...                0
1      0  I used to walk around the park with my coworke...                0
2      1  I used to walk around the park with my coworke...                1
3      0  I used to walk around the park with my coworke...                0
4      0  I usually give a gift to my girlfriend on Vale...                1


In [ ]:
# 정확도 계산
from sklearn.metrics import accuracy_score

true_labels = test_df['label']
accuracy = accuracy_score(true_labels, pred_labels)
print(f"Accuracy: {accuracy:.4f}")

Accuracy: 0.7330
